# Useful SQL Server Scripts

## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)

### **Server**
> -  Last known CheckDB (Q1)
> - SQL Server installation date - When was SQL installed (Q2)
> - SQL Server installation date - When was SQL installed to run against diffrent version of SQL 1) #Useful for registered servers (Q3)
### **Database info**
### **Table Info**
### **String manipulation**
> - Split out string to individual elements

>-  Last known **DBCC CHECKDB** (Q1)

In [ ]:
DBCC DBINFO('[Enter Database Name') WITH TABLERESULTS


>-  SQL Server Installation Date (Q2)

In [ ]:
SELECT create_date
FROM sys.server_principals
WHERE sid = 0x010100000000000512000000


> - Installation Dates (different versions of SQL )

In [ ]:
DECLARE @SQLServerInstanceVersion INT
SELECT  @SQLServerInstanceVersion=FLOOR(SUBSTRING(CAST(SERVERPROPERTY('ProductVersion')AS VARCHAR(10)),1,4)) 

SELECT @SQLServerInstanceVersion AS SQLVersion

IF @SQLServerInstanceVersion < 10 
BEGIN 
	SELECT  @@servername AS servername,CONVERT(DATETIME, createdate, 103) AS create_date FROM syslogins 
	WHERE   name = 'sa'
END
ELSE
BEGIN
	SELECT  @@ServerName as ServerName,CONVERT(DATETIME, create_date, 103) AS create_date FROM sys.server_principals WHERE sid = 0x010100000000000512000000
END


>- Top 10 records from all table


In [ ]:
EXEC sp_msforeachtable " SELECT '?' AS TableName ;SELECT top 10 * FROM ?"

Query Setting

>- Split out string to individual elements

In [ ]:
/* Split out from comma separted list */
DECLARE
    @in_string VARCHAR(MAX) ='A B C,2,6,7,8,8,',
    @delimeter VARCHAR(1) =','

DECLARE @list TABLE(tuple VARCHAR(100))



BEGIN
        WHILE LEN(@in_string) > 0
        BEGIN
            INSERT INTO @list(tuple)
            SELECT left(@in_string, charindex(@delimeter, @in_string+',') -1) as tuple
    
            SET @in_string = stuff(@in_string, 1, charindex(@delimeter, @in_string + @delimeter), '')
        end
     
END

SELECT * FROM @list

> - Add list of servers to be added to registered servers, basic adds in no seperate descriptions / future enhancement   

In [ ]:
/* Add list of Servers #SQL */

DECLARE @ServerGroupID INT

DECLARE @FolderName sysname ='MIH'
	  , @description NVARCHAR(2048) =''	--Server description
	  , @Name sysname 					--Display name of the server
	  , @ServerName sysname 			--Actual Name of the server

SELECT @ServerGroupID=Server_Group_ID FROM msdb.dbo.sysmanagement_shared_server_groups_internal WHERE Name=@FolderName



/* Split out from comma separted list */
DECLARE
    @in_string VARCHAR(MAX) ,
    @delimeter VARCHAR(1) =','

SET @in_string=
'FCL-SQL-01
,FCL-SQL-02
,FCL-SQL-02
'


DECLARE @list TABLE(ServerName VARCHAR(100))
BEGIN
        WHILE LEN(@in_string) > 0
        BEGIN
            INSERT INTO @list(ServerName)
            SELECT left(@in_string, charindex(@delimeter, @in_string+',') -1) as tuple
    
            SET @in_string = stuff(@in_string, 1, charindex(@delimeter, @in_string + @delimeter), '')
        end
     
END

--Any already added lets remove
DELETE FROM @list
WHERE ServerName  IN (SELECT server_name FROM msdb.dbo.[sysmanagement_shared_registered_servers_internal] WHERE server_group_id=@ServerGroupID)

DECLARE @server_id int

--Loop through and add in servers
WHILE (SELECT count(*) FROM @list) >1 
BEGIN

		SELECT TOP 1 @ServerName=ServerName FROM @list

		SET @Name =@ServerName 
	
		EXEC msdb.dbo.sp_sysmanagement_add_shared_registered_server @server_group_id=@ServerGroupID, @name=@name, @server_name=@ServerName, @description=@description, @server_type=0, @server_id=@server_id OUTPUT
		SELECT @ServerName AS ServerName, @server_id AS server_id

		
		DELETE FROM @list
		WHERE ServerName=@ServerName 


END

>- List of values sudo table on the fly 

In [ ]:
/* table of value on the fly */
SELECT * FROM (values ('1'), ('2'), ('3'), ('4'), ('5')) SomeData(Title)

## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)